<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/cargarROADMAP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Celda Cero: Inicialización de GEE
import ee
PROYECTO_ID = "gen-lang-client-0528470346"

try:
    # Intenta inicializar directamente con el proyecto.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Earth Engine inicializado.")

except Exception as e:
    # Si falla, es porque el token expiró. Ejecuta la autenticación de nuevo.
    print("❌ Error de inicialización. Ejecutando ee.Authenticate()...")
    ee.Authenticate()
    # Si la autenticación es exitosa, intenta inicializar de nuevo.
    ee.Initialize(project=PROYECTO_ID)
    print("✅ Autenticación y reinicialización exitosa.")

✅ Earth Engine inicializado.


In [2]:

# Celda 2: Obtención y Procesamiento de la Imagen Sentinel-2

import ee
# Asegúrate de que GEE esté inicializado (Celda Cero) antes de ejecutar esta.

# 1. Definir la ubicación de interés (Mérida, Venezuela)
lon = -71.146
lat = 8.587
punto_merida = ee.Geometry.Point(lon, lat)

# 2. Definir el rango de tiempo y Colección Sentinel-2
fecha_inicio = '2024-01-01'
fecha_fin = '2024-03-01'

# ⚠️ Filtro de nubes relajado (30%) para garantizar la existencia de imágenes
coleccion_s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(fecha_inicio, fecha_fin) \
    .filterBounds(punto_merida) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# 3. Crear el mosaico compuesto (media de píxeles menos nublados)
# La función .buffer(30000) es el radio circular de 30km alrededor de Mérida.
imagen_compuesta = coleccion_s2.median().clip(punto_merida.buffer(30000))

print("✅ Datos Sentinel-2 procesados (mosaico compuesto listo).")

✅ Datos Sentinel-2 procesados (mosaico compuesto listo).


In [4]:
 # Celda 3: Visualización con Google ROADMAP y Centrado Forzado

from IPython.display import display, Markdown
import geemap

# Parámetros de la ubicación de Mérida
lon = -71.146
lat = 8.587
zoom_level = 10

# 1. Parámetros de Visualización (Color Natural)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] # Bandas Sentinel-2: Red, Green, Blue
}

# 2. Crear el objeto Mapa (solo con el basemap estable 'ROADMAP')
m = geemap.Map(basemap='ROADMAP')

# ⚠️ 3. FORZAR el centrado del mapa después de su creación. ¡Esto es lo que garantiza Mérida!
m.setCenter(lon, lat, zoom_level)
m.add_basemap('OpenStreetMap')# 4. Añadir la capa de imagen satelital de GEE
m.addLayer(
    ee_object=imagen_compuesta,
    vis_params=vis_params,
    name='Sentinel-2 Compuesta (Color Natural)'
)

# 5. Mostrar el mapa
m.add_layer_control()
display(Markdown("### ✅ Resultado: Centrado Exitoso en Mérida"))
m

### ✅ Resultado: Centrado Exitoso en Mérida

Map(center=[8.587, -71.146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…